# This is just a notebook to visualise 1kHz filtered raw data

## Setup everything

### Import packages

In [1]:
import os
import sys
import quantities as pq
import numpy as np
import neo
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.widgets import Slider, Button, Cursor
import scipy
from scipy import interpolate
from scipy import fftpack
from scipy import signal
from open_ephys.analysis import Session

import mmap
import xarray as xr
import dask.array as da
#%matplotlib widget

from ephyviewer import mkQApp, MainViewer, TraceViewer
from ephyviewer import AnalogSignalSourceWithScatter


from ipyfilechooser import FileChooser
import ipywidgets as widgets

# add the Contrib dir that contains all tools developped by MB : mbTools.py
#sys.path.append(os.path.join(os.path.dirname(sys.path[0]),'python'))
#print(os.path.join(os.path.dirname(sys.path[0]),'python'))
import mbTools

### Import local config, create it if inexistant
All user-specific configuration (projects, defautl path to files...) are stored in a file named localConfig.ini in the python subfolder of AudreyHayLab repo. It is ignored by git so that it remains truely local. If the file does not exist at beginning, it is created with default values that can be modified at whishes.

In [2]:
config = mbTools.localConf()
rawDataPath = config['DATA']['path']
print(f'All raw data are expected to be found in the folder: {rawDataPath}')
analysisPath = config['ANALYSIS']['path']
print(f'All analysis will be saved in the folder: {analysisPath}')

Local config file loaded from localConfig.ini
All raw data are expected to be found in the folder: /Users/mb
All analysis will be saved in the folder: /Users/mb/Analysis


## Choose experiment
Select the experiment to display. If the experiment was already analyzed, a saved_dictionary.pkl was created and contains all necessary variables. Select this file. Otherwise select the raw data recording file.
>**If you have a file with channel mapping somewhere**, we should make sure it is properly translated into a dict.pkl

In [4]:
currentFile = None
%store -r currentFile
print(currentFile)
%store currentFile
theExpe = mbTools.expeConfigDict(currentFile)

/Users/mb/Analysis/AProject/OneOfItsSubProject/control/0/saved_dictionary.pkl
Local config file loaded from localConfig.ini


FileChooser(path='/Users/mb/Analysis/AProject/OneOfItsSubProject/control/0', filename='saved_dictionary.pkl', …

/Users/mb/Analysis/AProject/OneOfItsSubProject/projectConfig.pkl
Stored 'currentFile' (str)


### Possibility to change raw data path 
if for some reason the path to the raw data is wrong, you can update it here

In [ ]:
#print(rawDataPath)
if rawDataPath is not None:
    rawDirname, rawFN = os.path.split(rawDataPath)
    rfc = FileChooser(path=rawDirname, filename=rawFN,select_default=True, show_only_dirs = False, title = "<b>ePhys data</b>")
else:
    rfc = FileChooser(show_only_dirs = False, title = "<b>ePhys data</b>")
display(rfc)

# Sample callback function
def update_rawDataPath(chooser):
    global rawDataPath
    rawDataPath = chooser.selected
    #updateDict(currentFile, 'rawDataPath', rawDataPath)

# Register callback function
rfc.register_callback(update_rawDataPath)

## Load Data

### Map the whole data into memory

In [ ]:
rawDataDir, rawFileBaseName = os.path.split(rawDataPath)

if rawFileBaseName == "continuous.dat":
    All = np.memmap(rawDataPath, mode='r', dtype='int16')
    #All = np.fromfile(filename, dtype="int16")
    All = All.reshape(-1,numChanels)
elif rawFileBaseName.endswith(".npy"):
    All = np.load(rawDataPath, mmap_mode= 'r')

In [ ]:
filenameT = os.path.join(rawDataDir,"timestamps.npy")
if os.path.isfile(filenameT):
    Timestamps = np.load(filenameT)
    Timestamps.shape
    Timestamps = Timestamps*2000
    Timestamps = Timestamps.astype(int)

### Extract submatrix of interest

In [ ]:
start = 000000
end = All.shape[0]
combined = np.empty((end-start,0),np.int16)
channelLabels = []
for region in channelsMap:
    print(region, "->", channelsMap[region])
    if len([canal["canal"] for canal in channelsMap[region] if canal["status"]==2])>0:
        c2 = [canal["canal"] for canal in channelsMap[region] if canal["status"]==2][0]
        c1 = [canal["canal"] for canal in channelsMap[region] if canal["status"]==1][0]
        print("Getting differential signal of channel {} - channel {} for {}".format(c2,c1,region))
        channelLabels.append(region)
        combined = np.append(combined, All[start:end, c2, np.newaxis] - All[:, c1, np.newaxis], axis=1)
    elif len([canal["canal"] for canal in channelsMap[region] if canal["status"]==1])>0:
        c = [canal["canal"] for canal in channelsMap[region] if canal["status"]==1][0]
        print("Getting floating signal of channel {} for {}".format(c,region))
        combined = np.append(combined, All[start:end,c, np.newaxis], axis=1)
        channelLabels.append(region)

In [ ]:
%gui qt # allows the app to be closed clean and reopen
app = mkQApp()


sample_rate = 1000.
t_start = 0.

TTL = Timestamps

#create 2 familly scatters from theses 2 indexes
scatter_indexes = {0: TTL, 1: TTL}
#and asign them to some channels each
scatter_channels = {0: [0, 12], 1: [0, 1]}
source = AnalogSignalSourceWithScatter(All, sample_rate, t_start, scatter_indexes, scatter_channels)


#Create the main window that can contain several viewers
win = MainViewer()
view1 = TraceViewer.from_numpy(combined, sample_rate, t_start, 'Signals', channel_names=channelLabels)
#view1 = TraceViewer(source=source)
win.add_view(view1)

#Parameters can be set in script
view1.params['display_labels'] = True
view1.params['scale_mode'] = 'same_for_all'
view1.auto_scale()

cmap = matplotlib.colormaps["hsv"]#Wistia"]
nCh = len(view1.by_channel_params.children())
for ch in range(nCh):
    #view1.by_channel_params[f'ch{ch}', 'gain'] = 0.00002
    #view1.by_channel_params[f'ch{ch}', 'offset'] = 0.1
    view1.by_channel_params[f'ch{ch}', 'color'] = matplotlib.colors.to_hex(cmap(ch/nCh), keep_alpha=False)
    pass


#Run
win.show()
#app.exec()  #if commented, the app is shown and fonctionnal. Maybe detecting buttons. the Python icon doesn't close any better